# 02 – Inventory Optimization (EOQ / ROP / Safety Stock)
**Author:** Mousumi Paul | Feb 2025

## 1. Setup

In [ ]:
import pandas as pd, sys
sys.path.append('..')
from src.inventory_engine import (
    run_optimization, cost_savings_analysis,
    plot_stock_health, plot_cost_breakdown,
    plot_cost_savings, plot_eoq_curve
)
print('✅ Ready')

## 2. Run Optimization

In [ ]:
params = pd.read_csv('../data/raw/inventory_params.csv')
inv_df = run_optimization(params, ordering_cost=2500, z_score=1.65, lead_time_days=14)
inv_df[['Category','EOQ_Units','Safety_Stock','Reorder_Point','Days_of_Supply','Total_Inventory_Cost_INR','Alert_Status']]

## 3. Full Results

In [ ]:
inv_df.to_csv('../data/processed/inventory_results.csv', index=False)
inv_df

## 4. Cost Savings vs Fixed-Order Baseline

In [ ]:
savings = cost_savings_analysis(params, baseline_order_qty=1000)
print(savings.to_string(index=False))
savings.to_csv('../data/processed/cost_savings.csv', index=False)

## 5. Charts

In [ ]:
import os
os.makedirs('../outputs/charts', exist_ok=True)
plot_stock_health(inv_df,   save_path='../outputs/charts/stock_health.png')
plot_cost_breakdown(inv_df, save_path='../outputs/charts/cost_breakdown.png')
plot_cost_savings(savings,  save_path='../outputs/charts/cost_savings.png')
print('✅ Charts saved')

## 6. EOQ Cost Curves

In [ ]:
for _, row in params.iterrows():
    hc = row['Unit_Cost_INR'] * row['Holding_Cost_Pct']
    safe = row['Category'].replace(' & ','_').replace(' ','_')
    plot_eoq_curve(row['Category'], row['Annual_Demand'], 2500, hc,
                   save_path=f'../outputs/charts/eoq_curve_{safe}.png')
print('✅ EOQ curves saved')

## 7. SQL INSERT Generator

In [ ]:
from src.inventory_engine import to_sql_insert
sql_text = to_sql_insert(inv_df, '2025-02-01')
with open('../data/processed/insert_inventory_snapshot.sql','w') as f:
    f.write(sql_text)
print(sql_text[:500])